In [ ]:
from dash_app.backend.db_dictonary import load_datasets
%load_ext autoreload
%autoreload 2

db_dictonary = load_datasets()
db_dictonary

In [15]:
import psutil
from dash_app.backend.utils.prepared_date import add_maper_values_to_mapper_function
from datasets import load_dataset
from dash_app.backend.utils.object.dataset import DataSet

name = "clarin-pl/polemo2-official"
mapper_values= {1: 'negatywne', 3: 'dwuznaczne', 0: 'neutralne', 2: 'pozytywne'}
target_col = 'target'
dataDict = load_dataset(name)

mapper_function = add_maper_values_to_mapper_function(mapper_values, target_col)

dataDict = dataDict.map(mapper_function, num_proc=psutil.cpu_count(logical=True), load_from_cache_file=True)

No config specified, defaulting to: polemo2-official/all_text
Found cached dataset polemo2-official (/home/andrzej/.cache/huggingface/datasets/clarin-pl___polemo2-official/all_text/0.0.0/2b75fdbe5def97538e81fb120f8752744b50729a4ce09bd75132bfc863a2fd70)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/andrzej/.cache/huggingface/datasets/clarin-pl___polemo2-official/all_text/0.0.0/2b75fdbe5def97538e81fb120f8752744b50729a4ce09bd75132bfc863a2fd70/cache-b6a1b952987bead8.arrow
Loading cached processed dataset at /home/andrzej/.cache/huggingface/datasets/clarin-pl___polemo2-official/all_text/0.0.0/2b75fdbe5def97538e81fb120f8752744b50729a4ce09bd75132bfc863a2fd70/cache-6db3c2276b67bcd3.arrow
Loading cached processed dataset at /home/andrzej/.cache/huggingface/datasets/clarin-pl___polemo2-official/all_text/0.0.0/2b75fdbe5def97538e81fb120f8752744b50729a4ce09bd75132bfc863a2fd70/cache-a134f64418ef98a1.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'target'],
        num_rows: 6573
    })
    validation: Dataset({
        features: ['text', 'target'],
        num_rows: 823
    })
    test: Dataset({
        features: ['text', 'target'],
        num_rows: 820
    })
})

In [12]:
from datasets import load_dataset_builder
dataset_builder = load_dataset_builder(name)
dataset_builder.as_dataset()['train']

No config specified, defaulting to: polemo2-official/all_text


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'target'],
    num_rows: 6573
})

In [ ]:
merged_data = db_dictonary['połączone'].data
train = merged_data['train'].to_pandas()
validation = merged_data['validation'].to_pandas()


To unidecode

In [ ]:
import unidecode

mapper_target_value = {
    'negatywne': 0,
    'pozytywne': 1,
}

texts_org = train['token_tekst'].str.join(" ")
texts = texts_org.apply(unidecode.unidecode)

texts_val = validation['token_tekst'].str.join(" ")

target = list(map(lambda x: mapper_target_value[x], train['ocena_tekst']))
target_val = list(map(lambda x: mapper_target_value[x], validation['ocena_tekst']))

texts

Notes

In [ ]:
print(texts_org[texts_org.str.contains('눉gdybym')].iloc[0])
print(texts_org[texts.str.contains('zzerac')].iloc[0])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(encoding='ascii', strip_accents='ascii', min_df=1)
X = vectorizer.fit_transform(texts)
# print(X.toarray().sum().sum())
vectorizer.get_feature_names_out()

Model

In [ ]:
target.value_counts() / target.shape[0]

In [ ]:

from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

text_clf = Pipeline(
    [
        ("vect", CountVectorizer(strip_accents='ascii', min_df=2)),
        ("tfidf", TfidfTransformer()),
        # ("clf", LogisticRegression()),
        ('clf', MultinomialNB()),
    ]
)
text_clf.fit(texts, target)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'vect__min_df': [1, 2],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, scoring='f1')
gs_clf = gs_clf.fit(texts, target)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

text_clf = Pipeline(
    [
        ("vect", CountVectorizer(strip_accents='ascii', min_df=1, ngram_range=(1, 2))),
        ("tfidf", TfidfTransformer(use_idf=False)),
        ('clf', MultinomialNB(alpha =0.01)),
    ]
)
text_clf.fit(texts, target)

y_pred = text_clf.predict(texts_val)
y_pred

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn import metrics


def plot_and_print_score_model(target_val, y_pred):
    conf_matrix = confusion_matrix(y_true=target_val, y_pred=y_pred)

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i, s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

    plt.xlabel('Predictions', fontsize=18)
    plt.ylabel('Actuals', fontsize=18)
    plt.title('Confusion Matrix', fontsize=18)
    plt.show()

    print('Precision: %.3f' % precision_score(target_val, y_pred))  # Precision Score = TP / (FP + TP)
    print('Recall: %.3f' % recall_score(target_val, y_pred))  # Recall Score = TP / (FN + TP)
    print('Accuracy: %.3f' % accuracy_score(target_val, y_pred))  # Accuracy Score = (TP + TN)/ (TP + FN + TN + FP)
    print('F1 Score: %.3f' % f1_score(target_val,
                                      y_pred))  # F1 Score = 2* Precision Score * Recall Score/ (Precision Score + Recall Score/)
    print(metrics.classification_report(target_val, y_pred, target_names=mapper_target_value.keys()))


# plot_and_print_score_model(target_val, y_pred)


In [ ]:
from sklearn.linear_model import SGDClassifier, LogisticRegression

text_clf = Pipeline(
    [
        ("vect", CountVectorizer(strip_accents='ascii', min_df=1, ngram_range=(1, 2))),
        ("tfidf", TfidfTransformer(use_idf=False)),
         ('clf', LogisticRegression(class_weight='balance',max_iter=1000)),
    ])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'vect__min_df': [1, 2],
    'tfidf__use_idf': (True, False),
    'clf__C': (1,10,100)

}

gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, scoring='f1')
gs_clf = gs_clf.fit(texts, target)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))


In [ ]:
# from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

text_clf = Pipeline(
    [
        ("vect", CountVectorizer(strip_accents='ascii', min_df=2, ngram_range=(1, 2))),
        ("tfidf", TfidfTransformer(use_idf=False)),
        ('clf', LogisticRegression(C=10,class_weight='balance')),
    ])

text_clf.fit(texts, target)

y_pred = text_clf.predict(texts_val)
plot_and_print_score_model(target_val, y_pred)
